# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive import BanditPolicy
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mlops-capstone-project-hyperdrive'
project_folder = './mlops-capstone-project-folder'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
mlops-capstone-project-hyperdrive,MLOPS,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
found = False
key = "Bank_Loan"
description_text = "Bank Personal Loan Dataset for Udacity capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
    print('not found ... Please make sure its correct')
        


df = dataset.to_pandas_dataframe()
df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-demo-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [32]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000), "--max_iter": choice(40,80,120,130,200)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', compute_target=compute_target, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                          max_total_runs=10,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [33]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883
Web View: https://ml.azure.com/runs/HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883?wsid=/subscriptions/1db20fb5-44d4-452c-9f69-3e242db6cb9e/resourcegroups/ml-resource-group/workspaces/MLOPS&tid=482198bb-ae7b-4b25-8b7a-6d7f32faa083

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-05T05:08:27.319554][API][INFO]Experiment created<END>\n""<START>[2022-01-05T05:08:28.033547][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2022-01-05T05:08:28.657633][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883
Web View: https://ml.azure.com/runs/HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883?wsid=/subscriptions/1db20fb5-44d4-452c-9f69-3e242db6cb9e/resourcegroups/ml-resource-group/workspaces/MLOPS&tid=482198bb-ae7b-4b25-8b7a-6d7f32faa083



{'runId': 'HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883',
 'target': 'cluster-demo-1',
 'status': 'Completed',
 'startTimeUtc': '2022-01-05T05:08:27.092227Z',
 'endTimeUtc': '2022-01-05T05:31:06.334995Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1f6fc9bf-19fe-4a84-9d28-0603291485f4',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '55',
  'score': '0.908',
  'best_child_run_id': 'HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883_3',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlops5538635530.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883/azureml-logs/hyp

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [53]:
import joblib
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])
joblib.dump(value=best_run.id, filename="./outputs/best_hyperdrive_model.joblib")

Best Run Id:  HD_c0a66447-1ebf-4cbf-a113-bb5b7139c883_3

 Accuracy: 0.908

 Regularization Strength: 0.1

 Max Iterations: 130


['./outputs/best_hyperdrive_model.joblib']

In [54]:
best_run_metrics = best_run.get_metrics() # or other runs with runID
for metric_name in best_run_metrics:
     metric = best_run_metrics[metric_name]
     print(metric_name, metric)

Regularization Strength: 0.1
Max iterations: 130
Accuracy 0.908


In [57]:
#TODO: Save the best model
bestModel = best_run.register_model(model_name='HyperDrive_Capstone_Best_Accuracy', model_path='./outputs', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})
bestModel

Model(workspace=Workspace.create(name='MLOPS', subscription_id='1db20fb5-44d4-452c-9f69-3e242db6cb9e', resource_group='ml-resource-group'), name=HyperDrive_Capstone_Best_Accuracy, id=HyperDrive_Capstone_Best_Accuracy:5, version=5, tags={}, properties={'Accuracy': '0.908', 'Regularization Strength': '0.1', 'Max Iterations': '130'})

bestModel = best_run.register_model(model_path='outputs/model.pkl', model_name='HyperDrive_Capstone_Best_Accuracy',
                        tags={'Capstone HyperDrive AML Training tag':'Capstone Hyperdrive Auto ML'},
                        properties={'Accuracy': best_run_metrics['Accuracy']})

print(bestModel)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [61]:
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoringScript-hyperdrive.py')
#best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')
inference_config = InferenceConfig(entry_script='scoringScript-hyperdrive.py',
                                    environment=best_run.get_environment())
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1) # To Deploy the model to ACI with a default environment you can still customize the deploy configuration (i.e. the number of cores and amount of memory made available for the deployment) using the AciWebservice.deploy_configuration(). (https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice?view=azure-ml-py)
service = Model.deploy(ws, "joydeep-capstone-hyperdrive-endpoint", [bestModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

UserErrorException: UserErrorException:
	Message: File with path outputs/scoring_file_v_1_0_0.py was not found,
available files include: logs/azureml/18_azureml.log,outputs/best_capstone_project_hyperdrive_model.joblib,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/lifecycler/vm-bootstrapper.log,user_logs/std_log.txt.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path outputs/scoring_file_v_1_0_0.py was not found,\navailable files include: logs/azureml/18_azureml.log,outputs/best_capstone_project_hyperdrive_model.joblib,system_logs/cs_capability/cs-capability.log,system_logs/hosttools_capability/hosttools-capability.log,system_logs/lifecycler/execution-wrapper.log,system_logs/lifecycler/lifecycler.log,system_logs/lifecycler/vm-bootstrapper.log,user_logs/std_log.txt."
    }

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

